In [52]:
import os
import glob
import csv
import pandas as pd
import numpy as np

In [11]:
application_recors = "C:/Users/Lenovo/Desktop/SZKOLENIA MATEUSZ/dane/project_python/application_record.csv"
credit_records = "C:/Users/Lenovo/Desktop/SZKOLENIA MATEUSZ/dane/project_python/credit_record.csv"


In [33]:
df_application_records = pd.read_csv(application_recors)
df_credit_records = pd.read_csv(credit_records)

df_credit_records


,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


In [34]:
#is there are null data?
df_credit_records.info()

#there is no null data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 3 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   ID              1048575 non-null  int64 
 1   MONTHS_BALANCE  1048575 non-null  int64 
 2   STATUS          1048575 non-null  object
dtypes: int64(2), object(1)
memory usage: 24.0+ MB


In [37]:
#is there duplicated values?
sum(df_credit_records.duplicated())

#there is no duplicated values

0

In [46]:
#month qty in month balance colun

print(len(set(df_credit_records["MONTHS_BALANCE"])))

print(set(df_credit_records["STATUS"]))

61
{'C', '0', 'X', '5', '4', '1', '3', '2'}


In [59]:
#pivot credit cards records
credit_records_pivot_tb = df_credit_records.pivot(index = 'ID', columns = 'MONTHS_BALANCE', values = 'STATUS')
credit_records_pivot_tb.head()


MONTHS_BALANCE,-60,-59,-58,-57,-56,-55,-54,-53,-52,-51,...,-9,-8,-7,-6,-5,-4,-3,-2,-1,0
ID,,,,,,,,,,,,,,,,,,,,,
5001711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,X
5001712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,C,C,C,C,C,C,C,C,C
5001713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,X,X,X,X,X,X,X,X,X,X
5001714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,X,X,X,X,X,X,X,X,X,X
5001715,NaN,X,X,X,X,X,X,X,X,X,...,X,X,X,X,X,X,X,X,X,X


In [76]:
df_credit_records.head()

paid_off = credit_records_pivot_tb[credit_records_pivot_tb[:61] == "C"].count(axis=1)
no_credit_card = credit_records_pivot_tb[credit_records_pivot_tb[:61] == "X"].count(axis=1)
days_past_due_1_29 = credit_records_pivot_tb[credit_records_pivot_tb[:61] == "0"].count(axis=1)
days_past_due_30_59 = credit_records_pivot_tb[credit_records_pivot_tb[:61] == "1"].count(axis=1)
days_past_due_60_89 = credit_records_pivot_tb[credit_records_pivot_tb[:61] == "2"].count(axis=1)
days_past_due_90_119 = credit_records_pivot_tb[credit_records_pivot_tb[:61] == "3"].count(axis=1)
days_past_due_120_149 = credit_records_pivot_tb[credit_records_pivot_tb[:61] == "4"].count(axis=1)
days_past_due_over_150 = credit_records_pivot_tb[credit_records_pivot_tb[:61] == "5"].count(axis=1)
not_NaN_values = credit_records_pivot_tb[credit_records_pivot_tb[:61] != "NaN"].count(axis=1)

credit_records_pivot_tb["paid_off"] = paid_off
credit_records_pivot_tb["no_credit_card"] = no_credit_card
credit_records_pivot_tb["not NoN val"] = not_NaN_values
credit_records_pivot_tb["pastdue 1-29"] = days_past_due_1_29
credit_records_pivot_tb["pastdue 30-59"] = days_past_due_30_59
credit_records_pivot_tb["pastdue 60-89"] = days_past_due_60_89
credit_records_pivot_tb["pastdue 90-119"] = days_past_due_90_119
credit_records_pivot_tb["pastdue 120-149"] = days_past_due_120_149
credit_records_pivot_tb["over 150"] = days_past_due_over_150

credit_records_pivot_tb

MONTHS_BALANCE,-60,-59,-58,-57,-56,-55,-54,-53,-52,-51,...,0,paid_off,no_credit_card,not NoN val,pastdue 1-29,pastdue 30-59,pastdue 60-89,pastdue 90-119,pastdue 120-149,over 150
ID,,,,,,,,,,,,,,,,,,,,,
5001711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,X,0,1,7,3,0,0,0,0,0
5001712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,C,9,0,22,10,0,0,0,0,0
5001713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,X,0,22,25,0,0,0,0,0,0
5001714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,X,0,15,18,0,0,0,0,0,0
5001715,NaN,X,X,X,X,X,X,X,X,X,...,X,0,60,63,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5150482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0,0,0,0,0,0,0,0
5150483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,X,0,0,0,0,0,0,0,0,0
5150484,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,C,0,0,0,0,0,0,0,0,0
